## Item 25: Initialize Parent Classes with super

* The old way to initialize a parent class from a child class is to directly call the parent class's `__init__` method with the child instance.

In [ ]:
class MyBaseClass:
    def __init__(self, value):
        self.value = value
        
class MyChildClass(MyBaseClass):
    def __init__(self):
        MyBaseClass.__init__(self, 5)
        
    def times_two(self):
        return self.value * 2

In [ ]:
foo = MyChildClass()
assert foo.times_two() == 10

* This approach works fine for simple hierarchies but breaks down in many cases.
* If your class is affected by multiple inheritance (something to avoid in general):
    * See `Item 26`: Use Multiple Inheritance Only for Mix-in Utility Classes.
    * Calling the superclasses' `__init__` mothods directly can lead to unpredictable behavior.

* One problem is that the `__init__` call order isn't specified across all subclasses.

In [ ]:
class TimesTwo:
    def __init__(self):
        self.value *= 2
        

class PlusFive:
    def __init__(self):
        self.value += 5

In [ ]:
class OneWay(MyBaseClass, TimesTwo, PlusFive):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

In [ ]:
foo = OneWay(5)

In [ ]:
print("First ordering value is (5 * 2) + 5 =", foo.value)

* Here is another class that defines the same parent classes but in a different ordering.

In [ ]:
class AnotherWay(MyBaseClass, PlusFive, TimesTwo):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

In [ ]:
bar = AnotherWay(5)
print("Second ordering still is", bar.value)

* Another problem occurs with diamond inheritance.
* Diamond inheritance happens when a subclass inherits from two separate classes that have the same superclass somewhere in the hierarchy.
* Diamond inheritance causes the common superclass's `__init__` method to run miltiple times, causing unexpected behavior.

In [ ]:
# replaced by
class TimesSeven(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value *= 7


# replaced by
class PlusNine(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value += 9

* Define a child class that inherits from both of these classes, making `MyBaseClass` the top of the diamond.

In [ ]:
class ThisWay(TimesSeven, PlusNine):
    def __init__(self, value):
        TimesSeven.__init__(self, value)
        PlusNine.__init__(self, value)

In [ ]:
(5 * 7) + 9

In [ ]:
foo = ThisWay(5)
print("Should be (5 * 7) + 9 = 44 but is", foo.value)

* Problem

* The call to the second parent class's constructor, `PlusNine.__init__`, causes `self.value` to be reset back to 5 when `MyBaseClass.__init__` gets called the second time.
* To solve the problems, Python added the `super` built-in function and defined the MRO (method resolution order).
* The MRO standardizes which superclasses are initialized before others.
* It also ensures that common superclasses in diamond hierarchies are only run once.

In [ ]:
class MyBaseClass:
    def __init__(self, value):
        self.value = value
        

class TimesSevenCorrect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value *= 7
        
        
class PlusNineCorrect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value += 9        

* Now the top part of the diamond, `MyBaseClass.__init__`, is only run a single time.
* The other parent classes are run in the order specified in the class statement.

* Making calls to `super` with no arguments equivalent to calling `super` with `__class__` and self specified.

In [ ]:
class GoodWay(TimesSevenCorrect, PlusNineCorrect):
    def __init__(self, value):
        super().__init__(value)

In [ ]:
(7 * 5) + 9

In [ ]:
7 * (5 + 9)

In [ ]:
foo = GoodWay(5)
print("Should be 7 * (5 + 9) = 98 and is", foo.value)

* This order may seem backwards at first.
* This ordering matches what the MRO defines for this class.
* The MRO ordering is available on a class method called `mro`.

* Once the initialization reaches the top of the diamond, then all of the initialization methods actually do their work in the opposite order from how their `__init__` functions wre called.
    * `MyBaseClass.__init__` assigns the value to 5. 
    * `PlusNineCorrect.__init__` adds 9.
    * `TimesSevenCorrect.__init__` multiplies it by 7.

* MRO

In [ ]:
GoodWay.mro()

In [ ]:
mro_str = '\n'.join(repr(cls) for cls in GoodWay.mro())
print(mro_str)

* You should always use `super` because it's clear, concise, and always does the right thing.

In [ ]:
class ExplicitTrisect(MyBaseClass):
    def __init__(self, value):
        super(ExplicitTrisect, self).__init__(value)
        self.value /= 3

In [ ]:
ExplicitTrisect(9).value

In [ ]:
class AutomaticTrisect(MyBaseClass):
    def __init__(self, value):
        super(__class__, self).__init__(value)
        self.value /= 3   

In [ ]:
AutomaticTrisect(9).value

In [ ]:
class ImplicitTrisect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value /= 3

In [ ]:
ImplicitTrisect(9).value

In [ ]:
3 == 3.0

In [ ]:
assert ExplicitTrisect(9).value == 3
assert AutomaticTrisect(9).value == 3
assert ImplicitTrisect(9).value == 3

### Things to Remember

* Python's standard MRO (method resolution order) solves the problems of superclass initialization order and diamond inheritance.
* Always use the `super` built-in function to initialize parent classes.